## classmethod

In [14]:
from array import array
import math

class Vector2d:
    # __slots__ = ('__x', '__y')
    typecode = 'd'

    def __init__(self, x, y):
        # 언더바 두개를 사용하여 속성을 비공개로 만든다.
        self.__x = float(x)
        self.__y = float(y)

    # property 데코레이터는 프로퍼티의 getter 메서드를 나타낸다.
    # 이렇게하면 클래스의 속성들이 불변형이 된다.
    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
                bytes(array(self.typecode, self)))
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
        
    def angle(self):
        return math.atan2(self.y, self.x)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
        
    # @classmethod는 메서드가 호출되는 방식을 변경해서 클래스 자체를 첫번째 인수로 받게 만들며,
    # 아래 함수와 같이 대안 생성자를 구현하기 위해 주로 사용된다.
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
        

    

In [3]:
v1 = Vector2d(3, 4)

octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [8]:
v = v1.frombytes(octets)
v

AttributeError: 'Vector2d' object has no attribute 'frombytes'

In [9]:
v = Vector2d.frombytes(octets)
v

Vector2d(3.0, 4.0)

## classmethod vs staticmethod

In [10]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    @staticmethod
    def statmeth(*args):
        return args

In [11]:
Demo.klassmeth()

(__main__.Demo,)

In [12]:
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

In [13]:
Demo.statmeth()

()

In [14]:
Demo.statmeth('spam')

('spam',)

## 포맷된 출력

In [16]:
brl = 1/2.43
format(brl, '0.4f')

'0.4115'

In [23]:
f'1 BRL = {brl:.2f} USD'

'1 BRL = 0.41 USD'

In [24]:
format(42, 'b')

'101010'

In [25]:
format(2/3, '.1%')

'66.7%'

In [28]:
f'1 BRL = {brl:.2%} USD', f'{3:b}'

('1 BRL = 41.15% USD', '11')

In [30]:
from datetime import datetime
now = datetime.now()
f'{now:%H:%M:%S}'

'21:50:20'

In [32]:
f"It's now {now:%H:%M %p}"

"It's now 21:50 PM"

In [36]:
v1 = Vector2d(3, 4)

f'{v1:.2f}'

'(3.00, 4.00)'

In [40]:

f'{v1:.3ep}'

'<5.000e+00, 9.273e-01>'

## property

In [2]:
v1 = Vector2d(3, 4)
v1.x, v1.y

(3.0, 4.0)

In [5]:
v1.x = 7

AttributeError: can't set attribute

In [6]:
hash(v1)

7

## 파이썬의 비공개 속성과 보호된 속성

In [8]:
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [9]:
v1._Vector2d__x

3.0

## \_\_slot__ 클래스 속성으로 공간 절약하기

## 클래스 속성 오버라이드

In [15]:
v1 = Vector2d(1.1, 2.2)
dumped = bytes(v1)
dumped

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [16]:
len(dumped)

17

In [18]:
v1.typecode = 'f'
dumpf = bytes(v1)
dumpf, len(dumpf)

(b'f\xcd\xcc\x8c?\xcd\xcc\x0c@', 9)

In [19]:
class ShortVector2d(Vector2d):
    typecode = 'f'

sv =  ShortVector2d(1/11, 2/27)
sv

ShortVector2d(0.09090909090909091, 0.07407407407407407)

In [20]:
len(bytes(sv))

9